# Experimento: Sistema Hormonal Dinámico en LLMs

**Objetivo:** Comparar generación estática vs dinámica usando perfiles hormonales

**Contenido:**
1. Instalación y configuración
2. Carga de dataset de prompts
3. Definición de perfiles hormonales
4. Ejecución del experimento
5. Análisis de resultados
6. Visualizaciones

---

## 1. Instalación y Configuración

**IMPORTANTE:** Si tienes problemas de compatibilidad, reinicia el entorno antes de ejecutar estas celdas.

In [ ]:
# Configurar variables de entorno
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['TRANSFORMERS_NO_TRAINER'] = '1'

print("Variables de entorno configuradas")

In [ ]:
# Instalar el paquete
print("Instalando endocrine-llm... (puede tardar unos minutos)")
!pip install -q git+https://github.com/Nicolakorff/tfm-endocrine-llm.git@v0.5.0

import endocrine_llm
print(f"Versión instalada: {endocrine_llm.__version__}")

In [ ]:
# Imports necesarios
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.auto import tqdm
import json
import warnings
from scipy.stats import ttest_ind

from endocrine_llm import EndocrineModulatedLLM, HormoneProfile
from endocrine_llm.metrics import TextMetrics

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

print("Imports completados")

## 2. Preparación de Datos

Aquí puedes:
- **Opción A:** Subir tu propio archivo CSV de prompts
- **Opción B:** Usar un dataset de ejemplo generado automáticamente

In [ ]:
# Crear directorios necesarios
DATA_DIR = Path("data")
PROMPTS_DIR = DATA_DIR / "prompts"
RESULTS_DIR = DATA_DIR / "results" / "dynamic_experiment"

PROMPTS_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Directorios creados:")
print(f"- {PROMPTS_DIR}")
print(f"- {RESULTS_DIR}")

In [ ]:
# OPCIÓN A: Subir archivo CSV de prompts
# Descomentar estas líneas si subes un archivo:
# from google.colab import files
# uploaded = files.upload()
# prompts_df = pd.read_csv(list(uploaded.keys())[0])

# OPCIÓN B: Crear dataset de ejemplo
print("Creando dataset de ejemplo...")

example_prompts = [
    # Positivos
    ("I'm feeling happy and excited about today!", "positive"),
    ("This is wonderful news!", "positive"),
    ("I just achieved my goal!", "positive"),
    ("Everything is going perfectly well.", "positive"),
    ("I love spending time with my friends.", "positive"),

    # Negativos
    ("I'm feeling stressed about work.", "negative"),
    ("This is very confusing and uncertain.", "negative"),
    ("I'm worried about the future.", "negative"),
    ("Everything seems to be going wrong.", "negative"),
    ("I feel anxious and overwhelmed.", "negative"),

    # Neutros/Informativos
    ("I need help understanding this problem.", "neutral"),
    ("Can you explain how this works?", "neutral"),
    ("Tell me about your day.", "neutral"),
    ("What are the main features?", "neutral"),
    ("Please describe the process.", "neutral"),

    # Creativos
    ("Imagine a world where technology and nature coexist.", "creative"),
    ("Write a story about a robot learning emotions.", "creative"),
    ("Describe a futuristic city.", "creative"),
    ("What if animals could talk?", "creative"),
    ("Create a character with unique abilities.", "creative"),
]

prompts_df = pd.DataFrame(example_prompts, columns=['prompt', 'category'])

# Guardar para referencia
prompts_df.to_csv(PROMPTS_DIR / "prompts_dataset.csv", index=False)

print(f"Dataset creado: {len(prompts_df)} prompts")
print(f"\nCategorías:")
print(prompts_df['category'].value_counts())
print(f"\nEjemplos:")
print(prompts_df.head(3))

## 3. Inicialización del Modelo

In [ ]:
print("Inicializando modelo...")
model = EndocrineModulatedLLM("distilgpt2")
print(f"Modelo cargado en: {model.device}")

## 4. Definición de Perfiles Hormonales

Se prueban 7 perfiles diferentes:
- **3 estáticos** (baseline, creativo, empático)
- **4 dinámicos** (neutral, creativo, empático, adaptativo)

In [ ]:
profiles_to_test = {
    # ═══════════════════════════════════════════════════════════
    # PERFILES ESTÁTICOS (baseline)
    # ═══════════════════════════════════════════════════════════
    'baseline_static': HormoneProfile(
        dopamine=0.5, cortisol=0.5, oxytocin=0.5,
        adrenaline=0.5, serotonin=0.5,
        dynamic=False
    ),

    'creative_static': HormoneProfile(
        dopamine=0.9,      # Alta recompensa
        cortisol=0.3,      # Bajo estrés
        oxytocin=0.5,      # Neutral
        adrenaline=0.6,    # Energía moderada-alta
        serotonin=0.5,     # Neutral
        dynamic=False
    ),

    'empathic_static': HormoneProfile(
        dopamine=0.6,      # Moderada recompensa
        cortisol=0.4,      # Bajo estrés
        oxytocin=0.9,      # Alta empatía
        adrenaline=0.4,    # Baja energía
        serotonin=0.7,     # Alto bienestar
        dynamic=False
    ),

    # ═══════════════════════════════════════════════════════════
    # PERFILES DINÁMICOS
    # ═══════════════════════════════════════════════════════════
    'neutral_dynamic': HormoneProfile(
        dopamine=0.5, cortisol=0.5, oxytocin=0.5,
        adrenaline=0.5, serotonin=0.5,
        dynamic=True,
        learning_rate=0.15  # Tasa estándar
    ),

    'creative_dynamic': HormoneProfile(
        dopamine=0.9, cortisol=0.3, oxytocin=0.5,
        adrenaline=0.6, serotonin=0.5,
        dynamic=True,
        learning_rate=0.12  # Aprende más lento (más estable)
    ),

    'empathic_dynamic': HormoneProfile(
        dopamine=0.6, cortisol=0.4, oxytocin=0.9,
        adrenaline=0.4, serotonin=0.7,
        dynamic=True,
        learning_rate=0.12  # Aprende más lento
    ),

    'adaptive_dynamic': HormoneProfile(
        dopamine=0.5, cortisol=0.5, oxytocin=0.5,
        adrenaline=0.5, serotonin=0.5,
        dynamic=True,
        learning_rate=0.2   # Aprende MÁS RÁPIDO (más reactivo)
    ),
}

print(f"Perfiles definidos: {len(profiles_to_test)}")
print(f"- Estáticos: {sum(1 for p in profiles_to_test.values() if not p.dynamic)}")
print(f"- Dinámicos: {sum(1 for p in profiles_to_test.values() if p.dynamic)}")

# Mostrar tabla de perfiles
print("\nTabla de perfiles:")
profile_data = []
for name, prof in profiles_to_test.items():
    profile_data.append({
        'Perfil': name,
        'Tipo': 'Dinámico' if prof.dynamic else 'Estático',
        'LR': prof.learning_rate if prof.dynamic else '-',
        'Dopa': prof.dopamine,
        'Cort': prof.cortisol,
        'Oxy': prof.oxytocin,
        'Adre': prof.adrenaline,
        'Sero': prof.serotonin
    })

pd.DataFrame(profile_data)

## 5. Ejecución del Experimento

Para cada combinación de prompt × perfil:
- Se generan **3 repeticiones**
- Se registran las hormonas iniciales y finales
- Se calculan métricas de texto

In [ ]:
# Configuración del experimento
NUM_REPETITIONS = 3
MAX_TOKENS = 50
UPDATE_INTERVAL = 5

total_iterations = len(prompts_df) * len(profiles_to_test) * NUM_REPETITIONS

print("="*80)
print("INICIANDO EXPERIMENTO")
print("="*80)
print(f"\nPrompts: {len(prompts_df)}")
print(f"Perfiles: {len(profiles_to_test)}")
print(f"Repeticiones: {NUM_REPETITIONS}")
print(f"Total generaciones: {total_iterations}")
print(f"Tokens por generación: {MAX_TOKENS}")
print("\n" + "="*80 + "\n")

results = []

with tqdm(total=total_iterations, desc="Progreso") as pbar:
    for _, prompt_row in prompts_df.iterrows():
        prompt = prompt_row['prompt']
        category = prompt_row.get('category', 'general')

        for profile_name, profile in profiles_to_test.items():
            for gen_idx in range(NUM_REPETITIONS):
                try:
                    if profile.dynamic:
                        # ═══════════════════════════════════════
                        # GENERACIÓN DINÁMICA
                        # ═══════════════════════════════════════
                        result = model.generate_with_dynamic_hormones(
                            prompt=prompt,
                            initial_profile=profile.clone(),
                            max_new_tokens=MAX_TOKENS,
                            update_interval=UPDATE_INTERVAL,
                            return_hormone_trajectory=True
                        )

                        generated_text = result['generated_text']
                        final_profile = result['final_hormone_profile']
                        trajectory = result.get('hormone_trajectory', [])

                    else:
                        # ═══════════════════════════════════════
                        # GENERACIÓN ESTÁTICA
                        # ═══════════════════════════════════════
                        texts = model.generate_with_hormones(
                            prompt=prompt,
                            hormone_profile=profile,
                            max_new_tokens=MAX_TOKENS,
                            num_return_sequences=1
                        )

                        generated_text = texts[0]
                        final_profile = profile.to_dict()
                        trajectory = []

                    # Calcular métricas de texto
                    metrics = TextMetrics.compute_all(generated_text)

                    # Guardar resultado completo
                    results.append({
                        # Metadata
                        'prompt': prompt,
                        'prompt_category': category,
                        'profile_name': profile_name,
                        'is_dynamic': profile.dynamic,
                        'learning_rate': profile.learning_rate if profile.dynamic else 0.0,
                        'generation_idx': gen_idx,
                        'generated_text': generated_text,

                        # Perfil hormonal inicial
                        'init_dopamine': profile.dopamine,
                        'init_cortisol': profile.cortisol,
                        'init_oxytocin': profile.oxytocin,
                        'init_adrenaline': profile.adrenaline,
                        'init_serotonin': profile.serotonin,

                        # Perfil hormonal final
                        'final_dopamine': final_profile['dopamine'],
                        'final_cortisol': final_profile['cortisol'],
                        'final_oxytocin': final_profile['oxytocin'],
                        'final_adrenaline': final_profile['adrenaline'],
                        'final_serotonin': final_profile['serotonin'],

                        # Cambios hormonales (deltas)
                        'delta_dopamine': final_profile['dopamine'] - profile.dopamine,
                        'delta_cortisol': final_profile['cortisol'] - profile.cortisol,
                        'delta_oxytocin': final_profile['oxytocin'] - profile.oxytocin,
                        'delta_adrenaline': final_profile['adrenaline'] - profile.adrenaline,
                        'delta_serotonin': final_profile['serotonin'] - profile.serotonin,

                        # Magnitud total del cambio
                        'total_hormone_change': sum([
                            abs(final_profile['dopamine'] - profile.dopamine),
                            abs(final_profile['cortisol'] - profile.cortisol),
                            abs(final_profile['oxytocin'] - profile.oxytocin),
                            abs(final_profile['adrenaline'] - profile.adrenaline),
                            abs(final_profile['serotonin'] - profile.serotonin),
                        ]),

                        # Métricas de texto
                        **metrics,

                        # Info adicional
                        'num_updates': len(trajectory) if trajectory else 0
                    })

                except Exception as e:
                    print(f"\nError en generación: {e}")
                    print(f"Prompt: {prompt[:50]}...")
                    print(f"Perfil: {profile_name}")
                    continue

                pbar.update(1)

print("\n" + "="*80)
print("EXPERIMENTO COMPLETADO")
print("="*80)
print(f"\nGeneraciones exitosas: {len(results)} / {total_iterations}")

## 6. Guardar Resultados

In [ ]:
# Crear DataFrame de resultados
df_results = pd.DataFrame(results)

# Guardar CSV
csv_path = RESULTS_DIR / "dynamic_results.csv"
df_results.to_csv(csv_path, index=False)
print(f"Resultados guardados: {csv_path}")

# Guardar metadata del experimento
metadata = {
    'total_prompts': len(prompts_df),
    'total_profiles': len(profiles_to_test),
    'repetitions_per_combination': NUM_REPETITIONS,
    'total_generations': len(df_results),
    'max_tokens': MAX_TOKENS,
    'update_interval': UPDATE_INTERVAL,
    'prompt_categories': prompts_df['category'].value_counts().to_dict(),
    'profiles_tested': list(profiles_to_test.keys()),
    'static_profiles': [name for name, p in profiles_to_test.items() if not p.dynamic],
    'dynamic_profiles': [name for name, p in profiles_to_test.items() if p.dynamic],
}

metadata_path = RESULTS_DIR / "experiment_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"Metadata guardada: {metadata_path}")

# Mostrar primeras filas
print(f"\nPrimeras filas del dataset:")
df_results.head()

## 7. Análisis Preliminar de Resultados

In [ ]:
print("="*80)
print("ANÁLISIS: CAMBIOS HORMONALES (Solo perfiles dinámicos)")
print("="*80 + "\n")

dynamic_df = df_results[df_results['is_dynamic'] == True]

if len(dynamic_df) > 0:
    print("Cambios promedio por hormona:")
    print("-" * 80)

    for hormone in ['dopamine', 'cortisol', 'oxytocin', 'adrenaline', 'serotonin']:
        delta_col = f'delta_{hormone}'
        mean_change = dynamic_df[delta_col].mean()
        std_change = dynamic_df[delta_col].std()
        abs_mean = dynamic_df[delta_col].abs().mean()

        print(f"{hormone.capitalize():12s}: Δ = {mean_change:+.4f} (±{std_change:.4f}) | |Δ| = {abs_mean:.4f}")

    print(f"\nMagnitud total de cambio promedio: {dynamic_df['total_hormone_change'].mean():.4f}")
    print(f"Desviación estándar: {dynamic_df['total_hormone_change'].std():.4f}")

    # Cambios por perfil dinámico
    print("\n" + "-" * 80)
    print("Cambios por perfil dinámico:")
    print("-" * 80)

    for profile_name in dynamic_df['profile_name'].unique():
        profile_data = dynamic_df[dynamic_df['profile_name'] == profile_name]
        mean_total = profile_data['total_hormone_change'].mean()
        std_total = profile_data['total_hormone_change'].std()
        lr = profile_data['learning_rate'].iloc[0]

        print(f"{profile_name:25s} (LR={lr:.2f}): {mean_total:.4f} (±{std_total:.4f})")

else:
    print("No hay perfiles dinámicos en los resultados.")

In [ ]:
print("="*80)
print("ANÁLISIS: MÉTRICAS DE TEXTO")
print("="*80 + "\n")

print("Diversidad léxica (Distinct-2) por perfil:")
print("-" * 80)

for profile_name in profiles_to_test.keys():
    profile_df = df_results[df_results['profile_name'] == profile_name]
    if len(profile_df) > 0:
        mean_distinct = profile_df['distinct_2'].mean()
        std_distinct = profile_df['distinct_2'].std()
        is_dyn = "[DYN]" if profiles_to_test[profile_name].dynamic else "[STA]"

        print(f"{profile_name:25s} {is_dyn}: {mean_distinct:.4f} (±{std_distinct:.4f})")

print("\nLongitud promedio por perfil:")
print("-" * 80)

for profile_name in profiles_to_test.keys():
    profile_df = df_results[df_results['profile_name'] == profile_name]
    if len(profile_df) > 0:
        mean_length = profile_df['length'].mean()
        std_length = profile_df['length'].std()
        is_dyn = "[DYN]" if profiles_to_test[profile_name].dynamic else "[STA]"

        print(f"{profile_name:25s} {is_dyn}: {mean_length:.1f} (±{std_length:.1f}) tokens")

print("\nSentimiento promedio por perfil:")
print("-" * 80)

for profile_name in profiles_to_test.keys():
    profile_df = df_results[df_results['profile_name'] == profile_name]
    if len(profile_df) > 0:
        mean_sentiment = profile_df['sentiment_polarity'].mean()
        std_sentiment = profile_df['sentiment_polarity'].std()
        is_dyn = "[DYN]" if profiles_to_test[profile_name].dynamic else "[STA]"

        print(f"{profile_name:25s} {is_dyn}: {mean_sentiment:+.4f} (±{std_sentiment:.4f})")

In [ ]:
print("="*80)
print("COMPARACIÓN: ESTÁTICO vs DINÁMICO")
print("="*80 + "\n")

static_df = df_results[~df_results['is_dynamic']]
dynamic_df = df_results[df_results['is_dynamic']]

if len(static_df) > 0 and len(dynamic_df) > 0:

    # T-tests para significancia estadística
    print("Tests de significancia (t-test):")
    print("-" * 80)

    for metric in ['distinct_2', 'repetition_rate', 'sentiment_polarity', 'length']:
        if metric in df_results.columns:
            t_stat, p_val = ttest_ind(
                static_df[metric].dropna(),
                dynamic_df[metric].dropna()
            )

            sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else "ns"
            print(f"{metric:20s}: t={t_stat:+.3f}, p={p_val:.4f} {sig}")

    print("\n(*** p<0.001, ** p<0.01, * p<0.05, ns=no significativo)")

    # Comparación de medias
    print("\n" + "-" * 80)
    print("Comparación de medias:")
    print("-" * 80)

    print("\nDiversidad léxica (Distinct-2):")
    print(f"  Estático:  {static_df['distinct_2'].mean():.4f} (±{static_df['distinct_2'].std():.4f})")
    print(f"  Dinámico:  {dynamic_df['distinct_2'].mean():.4f} (±{dynamic_df['distinct_2'].std():.4f})")
    diff = dynamic_df['distinct_2'].mean() - static_df['distinct_2'].mean()
    print(f"  Diferencia: {diff:+.4f}")

    print("\nTasa de repetición:")
    print(f"  Estático:  {static_df['repetition_rate'].mean():.4f} (±{static_df['repetition_rate'].std():.4f})")
    print(f"  Dinámico:  {dynamic_df['repetition_rate'].mean():.4f} (±{dynamic_df['repetition_rate'].std():.4f})")
    diff = dynamic_df['repetition_rate'].mean() - static_df['repetition_rate'].mean()
    print(f"  Diferencia: {diff:+.4f}")

    print("\nPolaridad del sentimiento:")
    print(f"  Estático:  {static_df['sentiment_polarity'].mean():.4f} (±{static_df['sentiment_polarity'].std():.4f})")
    print(f"  Dinámico:  {dynamic_df['sentiment_polarity'].mean():.4f} (±{dynamic_df['sentiment_polarity'].std():.4f})")
    diff = dynamic_df['sentiment_polarity'].mean() - static_df['sentiment_polarity'].mean()
    print(f"  Diferencia: {diff:+.4f}")

else:
    print("No hay suficientes datos para comparar estático vs dinámico.")

## 8. Visualizaciones

In [ ]:
# Preparar datos para visualización
df_viz = df_results.copy()
df_viz['Type'] = df_viz['is_dynamic'].map({True: 'Dinámico', False: 'Estático'})

# Crear figura con múltiples subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Comparación Estático vs Dinámico', fontsize=16, fontweight='bold', y=1.00)

# 1. Diversidad léxica
ax = axes[0, 0]
sns.boxplot(data=df_viz, x='Type', y='distinct_2', ax=ax, palette='Set2')
ax.set_title('Diversidad Léxica (Distinct-2)', fontweight='bold')
ax.set_xlabel('Tipo de Perfil')
ax.set_ylabel('Distinct-2')
ax.grid(True, alpha=0.3)

# 2. Tasa de repetición
ax = axes[0, 1]
sns.boxplot(data=df_viz, x='Type', y='repetition_rate', ax=ax, palette='Set2')
ax.set_title('Tasa de Repetición', fontweight='bold')
ax.set_xlabel('Tipo de Perfil')
ax.set_ylabel('Repetition Rate')
ax.grid(True, alpha=0.3)

# 3. Sentimiento
ax = axes[1, 0]
sns.boxplot(data=df_viz, x='Type', y='sentiment_polarity', ax=ax, palette='Set2')
ax.set_title('Polaridad del Sentimiento', fontweight='bold')
ax.set_xlabel('Tipo de Perfil')
ax.set_ylabel('Sentiment Polarity')
ax.axhline(y=0, color='red', linestyle='--', alpha=0.5, linewidth=1)
ax.grid(True, alpha=0.3)

# 4. Longitud
ax = axes[1, 1]
sns.boxplot(data=df_viz, x='Type', y='length', ax=ax, palette='Set2')
ax.set_title('Longitud del Texto', fontweight='bold')
ax.set_xlabel('Tipo de Perfil')
ax.set_ylabel('Tokens')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'static_vs_dynamic_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Gráfico guardado: {RESULTS_DIR / 'static_vs_dynamic_comparison.png'}")

In [ ]:
# Visualización de cambios hormonales (solo dinámicos)
if len(dynamic_df) > 0:
    fig, ax = plt.subplots(figsize=(12, 6))

    dynamic_df['total_hormone_change'].hist(
        bins=30,
        ax=ax,
        color='#e74c3c',
        alpha=0.7,
        edgecolor='black'
    )

    ax.axvline(
        x=dynamic_df['total_hormone_change'].mean(),
        color='blue',
        linestyle='--',
        linewidth=2,
        label=f"Media = {dynamic_df['total_hormone_change'].mean():.3f}"
    )

    ax.set_xlabel('Cambio Hormonal Total', fontsize=12, fontweight='bold')
    ax.set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
    ax.set_title('Distribución de Cambios Hormonales (Perfiles Dinámicos)',
                 fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'hormone_changes_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()

    print(f"Gráfico guardado: {RESULTS_DIR / 'hormone_changes_distribution.png'}")

In [ ]:
# Comparación de diversidad léxica por perfil
fig, ax = plt.subplots(figsize=(14, 6))

profile_order = list(profiles_to_test.keys())
df_viz_profiles = df_viz[df_viz['profile_name'].isin(profile_order)]

sns.boxplot(
    data=df_viz_profiles,
    x='profile_name',
    y='distinct_2',
    hue='Type',
    order=profile_order,
    ax=ax,
    palette='Set2'
)

ax.set_xlabel('Perfil Hormonal', fontsize=12, fontweight='bold')
ax.set_ylabel('Distinct-2', fontsize=12, fontweight='bold')
ax.set_title('Diversidad Léxica por Perfil', fontsize=14, fontweight='bold')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.legend(title='Tipo', fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'diversity_by_profile.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Gráfico guardado: {RESULTS_DIR / 'diversity_by_profile.png'}")

## 9. Resumen Final

In [ ]:
print("="*80)
print(" RESUMEN DEL EXPERIMENTO")
print("="*80)

print(f"\nDATOS GENERADOS:")
print(f"- Total de generaciones: {len(df_results)}")
print(f"- Prompts únicos: {df_results['prompt'].nunique()}")
print(f"- Perfiles probados: {len(profiles_to_test)}")
print(f"  - Estáticos: {len(static_df) // NUM_REPETITIONS // len(prompts_df)}")
print(f"  - Dinámicos: {len(dynamic_df) // NUM_REPETITIONS // len(prompts_df)}")

print(f"\nHALLAZGOS PRINCIPALES:")

if len(static_df) > 0 and len(dynamic_df) > 0:
    print(f"\nDiversidad léxica:")
    diff_diversity = dynamic_df['distinct_2'].mean() - static_df['distinct_2'].mean()
    trend = "↑" if diff_diversity > 0 else "↓"
    print(f"    Dinámico {trend} Estático: {abs(diff_diversity):.4f}")

    print(f"\nRepetición:")
    diff_rep = dynamic_df['repetition_rate'].mean() - static_df['repetition_rate'].mean()
    trend = "↓" if diff_rep < 0 else "↑"
    print(f"    Dinámico {trend} Estático: {abs(diff_rep):.4f}")

if len(dynamic_df) > 0:
    print(f"\nCambios hormonales (dinámicos):")
    print(f"    Cambio total medio: {dynamic_df['total_hormone_change'].mean():.4f}")
    print(f"    Rango: [{dynamic_df['total_hormone_change'].min():.4f}, {dynamic_df['total_hormone_change'].max():.4f}]")

print(f"\nARCHIVOS GENERADOS:")
print(f"- {RESULTS_DIR / 'dynamic_results.csv'}")
print(f"- {RESULTS_DIR / 'experiment_metadata.json'}")
print(f"- {RESULTS_DIR / 'static_vs_dynamic_comparison.png'}")
print(f"- {RESULTS_DIR / 'hormone_changes_distribution.png'}")
print(f"- {RESULTS_DIR / 'diversity_by_profile.png'}")

print("\n" + "="*80)
print("EXPERIMENTO FINALIZADO")
print("="*80)

## 10. Descargar Resultados

Ejecuta la siguiente celda para descargar todos los archivos generados.

In [ ]:
# Comprimir resultados en un ZIP
import shutil

shutil.make_archive(
    'dynamic_experiment_results',
    'zip',
    RESULTS_DIR
)

print("Archivo ZIP creado: dynamic_experiment_results.zip")

# Descargar en Colab
from google.colab import files
files.download('dynamic_experiment_results.zip')

print("Descarga iniciada")